# LightGBM Hello World Example

In this notebook, we are going to mention Microsoft LightGBM framework on a simple data set. This includes both preprocessing steps and modelling parts. You may change the source data set and run this notebook again.

### Prerequisites

You need to install these packages by running the following command

pip install pandas numpy lightgbm graphviz matplotlib

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

To plot built decision tree you need to download graphviz from the following link

https://graphviz.gitlab.io/_pages/Download/Download_windows.html

You will specify the installed path in the following block. If you haven't installed graphviz, please set plotTree variable to False

In [ ]:
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin' #be sure where graphviz installed

In [ ]:
is_regression = False #set this True to run classification
plotTree = False #if you haven't installed graphviz set this to False

We are going to work on playing golf decision data set.

You can find the raw data set here: https://github.com/serengil/decision-trees-for-ml/tree/master/dataset

In [ ]:
dataset = pd.read_csv('C:/Users/IS96273/Desktop/decision tree/dataset/golf2.txt')

In [ ]:
dataset.head()

As seen loaded data set includes both continuous and string features

### Label encoding

LightGBM framework expects to convert categorical features to integer before constructing the dataset. That's why, we are going to apply label encoding to categorical features

Ref: https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

features = []; categorical_features = []

num_of_rows = dataset.shape[0]
num_of_columns = dataset.shape[1]
num_of_classes = 1 #default value is 1 for regression. we will update this for classification.

for i in range(0, num_of_columns):
    column_name = dataset.columns[i]
    column_type = dataset[column_name].dtypes
    
    if i != num_of_columns - 1: #skip target
        features.append(column_name)
    
    if column_type == 'object':
        #encode with sklearn
        le.fit(dataset[column_name])
        feature_classes = list(le.classes_)
        #print(feature_classes)
        
        encoded_feature = le.transform(dataset[column_name])
        dataset[column_name] = pd.DataFrame(encoded_feature)
        
        #encode with manually
        """feature_classes = dataset[column_name].unique()
        
        for j in range(len(feature_classes)):
            feature_class = feature_classes[j]
            print(feature_class," -> ",j,", ",end='')
                        
            dataset[column_name] = dataset[column_name].replace(feature_class, str(j))"""
        
        if i != num_of_columns - 1: #skip target
            categorical_features.append(column_name)
        
        if is_regression == False and i == num_of_columns - 1:
            num_of_classes = len(feature_classes)

In [ ]:
print("num_of_classes: ",num_of_classes)
print("features: ",features)
print("categorical features: ",categorical_features)

In [ ]:
dataset.head()

In [ ]:
target_name = dataset.columns[num_of_columns - 1] #target is the final column at the right on data set

y_train = dataset[target_name].values
x_train = dataset.drop(columns=[target_name]).values

print("input features:\n",x_train)
print("--------------------")
print("output:\n",y_train)

Now, we are going to create data set for LightGBM. We have already transformed categorical features to integer in previous steps. Here, we have to define categorical features. Otherwise, decision node will check instance's that feature greater than some threshold or less than the threshold. Suppose that feature is related to gender information and values are -1 for unknown, 0 for man and 1 for woman. In this case, decision node might check that gender information is greater than -1. This might cause a trouble. That's why, specifying categorical features is very important.

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train
    ,feature_name = features
    , categorical_feature = categorical_features
)

In [ ]:
params = {
    'task': 'train'
    , 'boosting_type': 'gbdt'
    , 'objective': 'regression' if is_regression == True else 'multiclass'
    , 'num_class': num_of_classes
    , 'metric': 'rmsle' if is_regression == True else 'multi_logloss'
    , 'min_data': 1
    #, 'learning_rate':0.1
    , 'verbose': -1
}

gbm = lgb.train(params, lgb_train, num_boost_round=50)

In [ ]:
predictions = gbm.predict(x_train)

print(predictions)
"""for i in predictions:
    print(np.argmax(i))"""

In [ ]:
for index, instance in dataset.iterrows():
    actual = instance[target_name]
    
    if is_regression == True:
        prediction = round(predictions[index])
    else: #classification
        prediction = np.argmax(predictions[index])
    
    print((index+1),". actual= ",actual,", prediction= ",prediction)

In [ ]:
if plotTree == True:
    
    fig_size = [30, 20]
    plt.rcParams["figure.figsize"] = fig_size

    ax = lgb.plot_tree(gbm)
    plt.show()
    
    #ax = lgb.plot_importance(gbm, max_num_features=10)
    #plt.show()